In [26]:
import csv
import datetime
import pandas as pd
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


In [27]:
# Read the CSV file into a pandas DataFrame
df_og = pd.read_csv('modified_weatherAUS.csv')

*Iceberg query*

In [28]:
'''the results to only include documents from the past 5 years, 
    orders the results by MaxTemp in descending order, 
    and returns only the top 10 results.'''

# Open the CSV file
with open('modified_weatherAUS.csv') as csvfile:
    reader = csv.DictReader(csvfile)

    # Determine the date range for the past 5 years
    dates = [datetime.datetime.strptime(row['Date'], '%Y-%m-%d') for row in reader]
    max_date = max(dates)
    min_date = max_date - datetime.timedelta(days=365 * 5)

    # Reset the reader to the beginning of the file
    csvfile.seek(0)
    reader = csv.DictReader(csvfile)

    # Filter the rows based on the date range
    filtered_rows = []
    for row in reader:
        row_date = datetime.datetime.strptime(row['Date'], '%Y-%m-%d')
        if min_date <= row_date <= max_date:
            filtered_rows.append(row)

    # Sort the filtered rows by MaxTemp in descending order
    sorted_rows = sorted(filtered_rows, key=lambda x: float(x['MaxTemp']), reverse=True)

    # Return only the top 10 rows
    top_10_rows = sorted_rows[:10]

    # Print out the results
    for row in top_10_rows:
        print(row)



{'id': '11732', 'Date': '2014-01-03', 'Location': 'Moree', 'MinTemp': '28.3', 'MaxTemp': '47.3', 'Rainfall': '0.0', 'WindGustDir': 'WSW', 'WindGustSpeed': '61.0', 'WindDir9am': 'N', 'WindDir3pm': 'WNW', 'WindSpeed9am': '33.0', 'WindSpeed3pm': '33.0', 'Humidity9am': '36.0', 'Humidity3pm': '5.0', 'Pressure9am': '1001.9', 'Pressure3pm': '995.9', 'Temp9am': '33.4', 'Temp3pm': '45.8', 'RainToday': 'No', 'RainTomorrow': 'No', 'AvgTemp': '37.8', 'AvgWind': '42.333333333333336', 'AvgRainfall': '0.0', 'AvgHumidity': '20.5', 'AvgPressure': '998.9'}
{'id': '12611', 'Date': '2017-02-12', 'Location': 'Moree', 'MinTemp': '27.8', 'MaxTemp': '47.3', 'Rainfall': '0.0', 'WindGustDir': 'SSW', 'WindGustSpeed': '67.0', 'WindDir9am': 'NNE', 'WindDir3pm': 'NW', 'WindSpeed9am': '13.0', 'WindSpeed3pm': '28.0', 'Humidity9am': '30.0', 'Humidity3pm': '10.0', 'Pressure9am': '1003.9', 'Pressure3pm': '998.1', 'Temp9am': '32.9', 'Temp3pm': '46.7', 'RainToday': 'No', 'RainTomorrow': 'No', 'AvgTemp': '37.55', 'AvgWind'

windowing query

In [29]:
'''calculates the rolling average maximum temperature 
    over a 7-day window for each location'''

#copy of df_og
df = df_og.copy()

# Convert the 'Date' column to a datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Sort the DataFrame by date and location
df = df.sort_values(by=['Location', 'Date'])

# Calculate the rolling average maximum temperature over a 7-day window for each location
df['RollingAvgMaxTemp'] = df.groupby('Location')['MaxTemp'].rolling(window=7).mean().reset_index(drop=True)

# Print out the results
print(df[['Location', 'Date', 'MaxTemp', 'RollingAvgMaxTemp']])


       Location       Date  MaxTemp  RollingAvgMaxTemp
75811  Adelaide 2008-07-01     15.7          13.942857
75812  Adelaide 2008-07-02     15.8          13.714286
75813  Adelaide 2008-07-03     15.1          13.400000
75814  Adelaide 2008-07-04     15.9          13.257143
75815  Adelaide 2008-07-07     11.2          13.400000
...         ...        ...      ...                ...
87111   Woomera 2017-06-20     19.7          21.100000
87112   Woomera 2017-06-21     19.9          20.371429
87113   Woomera 2017-06-22     19.0          20.228571
87114   Woomera 2017-06-23     22.5          20.171429
87115   Woomera 2017-06-24     20.9          20.642857

[112960 rows x 4 columns]


using window close

In [30]:
'''retrieve the previous and next year's maximum temperature for each location'''

#copy of df_og
df = df_og.copy()

# Convert the Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Sort the DataFrame by Location and Date
df = df.sort_values(by=['Location', 'Date'])

# Group the DataFrame by Location
grouped_df = df.groupby('Location')

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Location', 'Date', 'MaxTemp', 'PrevMaxTemp', 'NextMaxTemp'])

# Loop through each location in the grouped DataFrame
for location, location_df in grouped_df:

    # Loop through each row in the location DataFrame
    for index, row in location_df.iterrows():

        # Get the date and maximum temperature for the current row
        date = row['Date']
        max_temp = row['MaxTemp']

        # Get the previous year's maximum temperature
        prev_year = date.year - 1
        prev_year_df = location_df[location_df['Date'].dt.year == prev_year]
        if not prev_year_df.empty:
            prev_max_temp = prev_year_df['MaxTemp'].max()
        else:
            prev_max_temp = None

        # Get the next year's maximum temperature
        next_year = date.year + 1
        next_year_df = location_df[location_df['Date'].dt.year == next_year]
        if not next_year_df.empty:
            next_max_temp = next_year_df['MaxTemp'].max()
        else:
            next_max_temp = None

        # Add a new row to the results DataFrame
        results_df = results_df.append({
            'Location': location,
            'Date': date,
            'MaxTemp': max_temp,
            'PrevMaxTemp': prev_max_temp,
            'NextMaxTemp': next_max_temp
        }, ignore_index=True)

# Print out the results
print(results_df)



        Location       Date  MaxTemp PrevMaxTemp NextMaxTemp
0       Adelaide 2008-07-01     15.7         NaN        45.7
1       Adelaide 2008-07-02     15.8         NaN        45.7
2       Adelaide 2008-07-03     15.1         NaN        45.7
3       Adelaide 2008-07-04     15.9         NaN        45.7
4       Adelaide 2008-07-07     11.2         NaN        45.7
...          ...        ...      ...         ...         ...
112955   Woomera 2017-06-20     19.7        43.9        None
112956   Woomera 2017-06-21     19.9        43.9        None
112957   Woomera 2017-06-22     19.0        43.9        None
112958   Woomera 2017-06-23     22.5        43.9        None
112959   Woomera 2017-06-24     20.9        43.9        None

[112960 rows x 5 columns]


drill down

In [31]:
'''retrieves all weather data where the location is "Sydney" and the maximum temperature is greater than 
    or equal to 30'''

#copy of df_og
df = df_og.copy()

# Filter the data by location and maximum temperature
filtered_data = df[(df['Location'] == 'Sydney') & (df['MaxTemp'] >= 30)]

# Print the filtered data
print(filtered_data)


          id        Date Location  MinTemp  MaxTemp  Rainfall WindGustDir  \
20467  20468  2010-11-14   Sydney     20.5     30.4       0.0         NNE   
20488  20489  2010-12-09   Sydney     22.0     30.7       0.0           N   
20491  20492  2010-12-12   Sydney     17.8     30.8       0.0         SSW   
20503  20504  2010-12-26   Sydney     20.8     31.3       0.8         SSE   
20523  20524  2011-01-15   Sydney     22.0     30.4       0.6         NNE   
...      ...         ...      ...      ...      ...       ...         ...   
22566  22567  2017-02-10   Sydney     23.7     37.5       0.0         SSE   
22567  22568  2017-02-11   Sydney     23.9     36.2       0.0         SSW   
22568  22569  2017-02-12   Sydney     24.5     31.9       0.0         SSE   
22572  22573  2017-02-16   Sydney     19.5     30.0       0.0         NNE   
22606  22607  2017-03-22   Sydney     22.3     31.9      25.6         WSW   

       WindGustSpeed WindDir9am WindDir3pm  ...  Pressure3pm  Temp9am  \
20

roll up

In [32]:
'''retrieves the average maximum temperature 
    for each location '''

#copy of df_og
df = df_og.copy()

# Group the DataFrame by location and calculate the mean of the maximum temperature
avg_max_temp = df.groupby('Location')['MaxTemp'].mean()

# Print the result
print(avg_max_temp)


Location
Adelaide            23.068540
Albury              23.411557
AliceSprings        29.588047
BadgerysCreek       24.238775
Ballarat            18.345432
Bendigo             22.019049
Brisbane            26.463874
Cairns              29.534253
Canberra            21.061290
Cobar               25.732597
CoffsHarbour        23.934439
Dartmoor            20.146972
Darwin              32.541931
GoldCoast           25.712159
Hobart              17.903658
Katherine           34.971194
Launceston          19.515465
Melbourne           20.974384
MelbourneAirport    20.502352
Mildura             24.986814
Moree               27.045150
MountGambier        19.914033
Nhil                22.579183
NorahHead           22.550592
NorfolkIsland       21.785894
Nuriootpa           21.929498
PearceRAAF          26.556957
Perth               25.192558
PerthAirport        25.605748
Portland            18.083877
Richmond            24.887512
Sale                20.339876
Sydney              23.293673
S

slice

In [33]:
'''selects documents where the temperature (which is the average of min, max temp at 3am and temps at 9am) 
    is greater than 30 degrees'''

#copy of df_og
df = df_og.copy()

# calculate temperature
temperature = (df['MinTemp'] + df['MaxTemp'] + df['Temp9am'] + df['Temp3pm'])/4

# select data where temperature is greater than 30
df_filtered = df[temperature > 30].copy()
print(df_filtered)


            id        Date Location  MinTemp  MaxTemp  Rainfall WindGustDir  \
41          42  2009-01-14   Albury     17.4     43.0       0.0          NW   
47          48  2009-01-20   Albury     18.6     39.9       0.0         NNW   
48          49  2009-01-21   Albury     19.3     38.1       0.8         NNW   
55          56  2009-01-28   Albury     20.4     40.7       0.0           N   
56          57  2009-01-29   Albury     21.8     41.5       0.0          SW   
...        ...         ...      ...      ...      ...       ...         ...   
112870  112871  2017-03-25    Uluru     21.3     40.6       0.0           E   
112871  112872  2017-03-26    Uluru     18.4     41.6       0.0         NNW   
112872  112873  2017-03-27    Uluru     25.3     37.6       0.0         SSE   
112873  112874  2017-03-28    Uluru     24.1     37.0       0.0           E   
112874  112875  2017-03-29    Uluru     25.2     34.7       0.0         SSE   

        WindGustSpeed WindDir9am WindDir3pm  ...  P

dice

In [34]:
'''selects documents where the location is "Albury" 
    and the minimum temperature is less than or equal to 10 degrees'''

#copy of df_og
df = df_og.copy()

# select documents where location is "Albury" and min temp is less than or equal to 10 degrees
selected_docs = df[(df['Location'] == 'Albury') & (df['MinTemp'] <= 10)]

# return the selected documents as a new dataframe
new_df = pd.DataFrame(selected_docs)

# print the new dataframe
print(new_df)


        id        Date Location  MinTemp  MaxTemp  Rainfall WindGustDir  \
1        2  2008-12-02   Albury      7.4     25.1       0.0         WNW   
3        4  2008-12-04   Albury      9.2     28.0       0.0          NE   
7        8  2008-12-08   Albury      7.7     26.7       0.0           W   
8        9  2008-12-09   Albury      9.7     31.9       0.0         NNW   
17      18  2008-12-20   Albury      9.8     25.6       0.0         SSE   
...    ...         ...      ...      ...      ...       ...         ...   
2435  2436  2017-06-16   Albury      3.2     12.9       0.2         ENE   
2436  2437  2017-06-18   Albury      1.0     17.0       0.0           W   
2437  2438  2017-06-20   Albury      1.2     14.9       0.2         ESE   
2438  2439  2017-06-23   Albury      1.1     11.9       0.0          SE   
2439  2440  2017-06-24   Albury      1.1     14.1       0.2         WSW   

      WindGustSpeed WindDir9am WindDir3pm  ...  Pressure3pm  Temp9am  Temp3pm  \
1              44.

combination of olap

In [35]:
'''selects documents where the location is either "Albury" or "Melbourne", 
    and the minimum temperature is less than or equal to 10 degrees'''

#copy of df_og
df = df_og.copy()

# select rows where location is either "Albury" or "Melbourne"
# and minimum temperature is less than or equal to 10 degrees
selected_data = df[(df['Location'] == 'Albury') | (df['Location'] == 'Melbourne')]
selected_data = selected_data[selected_data['MinTemp'] <= 10]

# print the selected data
print(selected_data)


          id        Date   Location  MinTemp  MaxTemp  Rainfall WindGustDir  \
1          2  2008-12-02     Albury      7.4     25.1       0.0         WNW   
3          4  2008-12-04     Albury      9.2     28.0       0.0          NE   
7          8  2008-12-08     Albury      7.7     26.7       0.0           W   
8          9  2008-12-09     Albury      9.7     31.9       0.0         NNW   
17        18  2008-12-20     Albury      9.8     25.6       0.0         SSE   
...      ...         ...        ...      ...      ...       ...         ...   
51915  51916  2017-06-21  Melbourne      7.8     13.5       0.6         NNW   
51916  51917  2017-06-22  Melbourne      6.7     14.1       0.2           N   
51917  51918  2017-06-23  Melbourne      7.4     14.9       0.0         NNW   
51918  51919  2017-06-24  Melbourne      8.3     12.3       1.4          SW   
51919  51920  2017-06-25  Melbourne      8.6     14.3       0.0          NW   

       WindGustSpeed WindDir9am WindDir3pm  ...  Pr